In [83]:
import spacy
from kiwipiepy import Kiwi 
import re

In [84]:
nlp = spacy.load("ko_core_news_lg")

In [85]:
kiwi = Kiwi()

In [86]:
#decice when to scan deeply for sensitive keyword
SENSITIVE_KEYWORD = [
    "도메인",
    "IP  주소",
    "예산",
    "비밀번호"
]

In [87]:
#mask target with regex when in keyword have this pattern
PASSWORD_REGEX = r"\b[A-Za-z0-9@#$%^&+=!]{8,}\b"
IP_REGEX = r"\b(?:\d{1,3}\.){3}\d{1,3}\b"
DOMAIN_REGEX = r"\b[a-zA-Z0-9.-]+\.(?:com|net|org|kr|io)\b"


In [88]:
#declare ner to mask
SENSITIVE_NER_LABELS = [
    "PC",
    "OG",
    "LC"
]


In [89]:
#kiwi processing
def kiwi_preprocess(text):
    tokens = kiwi.tokenize(text)
    return " ".join([t.form for t in tokens])


In [90]:
def is_sensitive_context(text):
    return any(k in text for k in SENSITIVE_KEYWORD)


In [91]:
def extract_regex_entities(text):
    entities = []
    entities += re.findall(PASSWORD_REGEX, text)
    entities += re.findall(IP_REGEX, text)
    entities += re.findall(DOMAIN_REGEX, text)
    return entities


In [92]:
#extract ner
def extract_ner_entities(text):
    doc = nlp(text)
    return [
        ent.text
        for ent in doc.ents
        if ent.label_ in SENSITIVE_NER_LABELS
    ]


In [93]:
#mask data
def mask_text(text, targets):
    masked_text = text
    for t in set(targets):
        masked_text = masked_text.replace(t, "*" * len(t))
    return masked_text


In [94]:
def dlp_scan(text):
    # 1. Preprocess
    clean_text = kiwi_preprocess(text)

    # 2. Context check
    if not is_sensitive_context(clean_text):
        return text, False

    # 3. Extract sensitive targets
    regex_targets = extract_regex_entities(clean_text)
    ner_targets = extract_ner_entities(clean_text)

    all_targets = regex_targets + ner_targets

    if not all_targets:
        return text, False

    # 4. Mask
    masked_text = mask_text(clean_text, all_targets)

    return masked_text, True


In [95]:
text1 = "김민수의 계정 비밀번호는 TestAccount2026이다."

result, flagged = dlp_scan(text1)
print(flagged)
print(result)


True
김민수 의 계정 비밀번호 는 *********** 2026 이 다 .


In [96]:
def dlp_scan_debug(text):
    print("ORIGINAL:", text)

    clean_text = kiwi_preprocess(text)
    print("KIWI:", clean_text)

    print("Context:", is_sensitive_context(clean_text))

    regex_targets = extract_regex_entities(clean_text)
    print("Regex targets:", regex_targets)

    ner_targets = extract_ner_entities(clean_text)
    print("NER targets:", ner_targets)

    all_targets = regex_targets + ner_targets
    print("ALL targets:", all_targets)

    if not all_targets:
        print("❌ Nothing to mask")
        return text, False

    masked = mask_text(clean_text, all_targets)
    print("MASKED:", masked)

    return masked, True


In [97]:
text = "김민수의 계정 비밀번호는 TestAccount2026이다."
dlp_scan_debug(text)


ORIGINAL: 김민수의 계정 비밀번호는 TestAccount2026이다.
KIWI: 김민수 의 계정 비밀번호 는 TestAccount 2026 이 다 .
Context: True
Regex targets: ['TestAccount']
NER targets: []
ALL targets: ['TestAccount']
MASKED: 김민수 의 계정 비밀번호 는 *********** 2026 이 다 .


('김민수 의 계정 비밀번호 는 *********** 2026 이 다 .', True)

In [104]:
from kiwipiepy import Kiwi
import spacy
kiwi = Kiwi()
text = "김민수는 서울대학교에 재학 중입니다."

tokens = kiwi.tokenize(text)
for t in tokens:
    print(t.form, t.tag)

김민수 NNP
는 JX
서울대 NNP
학교 NNG
에 JKB
재학 NNG
중 NNB
이 VCP
ᆸ니다 EF
. SF


In [105]:
nlp = spacy.load("ko_core_news_lg")
clean_text = kiwi_preprocess(text)
doc = nlp(clean_text)

for ent in doc.ents:
    print(ent.text, ent.label_)

김민수 PS
서울대 OG


In [106]:
def extract_ps_entities(text):
    doc = nlp(text)
    return [
        ent.text
        for ent in doc.ents
        if ent.label_ == "PS"
    ]

In [107]:
#mask data
def mask_text(text, targets):
    masked_text = text
    for t in set(targets):
        masked_text = masked_text.replace(t, "*" * len(t))
    return masked_text


In [108]:
text = "김민수는 서울대에 재학 중이다."

clean_text = kiwi_preprocess(text)

ps_targets = extract_ps_entities(clean_text)
masked = mask_text(clean_text, ps_targets)

print(masked)


*** 는 서울대 에 재학 중 이 다 .
